<a href="https://colab.research.google.com/github/trachtok/dspracticum2020_data/blob/main/assignment05/assignmen05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 05

## Load libraries

In [24]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.utils import to_categorical
from google.colab import drive

from tensorflow.keras.preprocessing import image_dataset_from_directory

In [25]:
!pip install split-folders
import splitfolders

## Prepare input data

+ Mount Google Discs

In [4]:
drive.mount('/content/drive')

Mounted at /content/drive


+ check what data we have, how many images in each class?

In [101]:
!ls '/content/drive/My Drive/assignment05_data/batman' | wc -l
!ls '/content/drive/My Drive/assignment05_data/superman' | wc -l
!ls '/content/drive/My Drive/assignment05_data/spiderman' | wc -l

256
289
284


+ to effective usage of `ImageDataGenerator()` it would be better to have 2 folders: training and validation a in both specific subset of images for classes batman, superman & spiderman
+ to obtain such order, we can use package `splitfolders`

In [102]:
splitfolders.ratio("/content/drive/My Drive/assignment05_data", output="/content/drive/My Drive/assignment05_data/prepared_data", seed=1337, ratio=(.8, .2)) 



Copying files: 0 files [00:00, ? files/s]

Copying files: 6 files [00:00, 58.41 files/s]

Copying files: 19 files [00:00, 60.11 files/s]

Copying files: 31 files [00:00, 70.12 files/s]

Copying files: 37 files [00:00, 52.61 files/s]

Copying files: 47 files [00:00, 61.22 files/s]

Copying files: 54 files [00:00, 49.43 files/s]

Copying files: 64 files [00:01, 57.80 files/s]

Copying files: 73 files [00:01, 49.85 files/s]

Copying files: 86 files [00:01, 61.02 files/s]

Copying files: 94 files [00:01, 53.69 files/s]

Copying files: 108 files [00:01, 59.32 files/s]

Copying files: 121 files [00:01, 70.72 files/s]

Copying files: 130 files [00:01, 69.16 files/s]

Copying files: 143 files [00:02, 80.29 files/s]

Copying files: 153 files [00:02, 66.31 files/s]

Copying files: 162 files [00:02, 64.84 files/s]

Copying files: 173 files [00:02, 73.86 files/s]

Copying files: 182 files [00:02, 60.26 files/s]

Copying files: 195 files [00:02, 71.72 files/s]

Copying files: 204 files [00:02, 70

In [119]:
!ls "/content/drive/My Drive/assignment05_data/prepared_data"
!ls "/content/drive/My Drive/assignment05_data/prepared_data/train"
!ls "/content/drive/My Drive/assignment05_data/prepared_data/val"

train  val
batman	superman
batman	superman


In [120]:
train_dir = "/content/drive/My Drive/assignment05_data/prepared_data/train"
validation_dir = "/content/drive/My Drive/assignment05_data/prepared_data/val"

# MobileNetV2
+ download MobileNetV2 model without the top layer

In [91]:
BATCH_SIZE = 32
EPOCHS = 5
IMG_SIZE = (160, 160)
IMG_SHAPE = IMG_SIZE + (3,)

In [121]:
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')
# base_model.summary()

# Transfer learning
+ using already trained model, first freeze all layers

In [122]:
for layer in base_model.layers:
    layer.trainable = False

+ then add pooling layer and dropout, finallu put on top `softmax` for categorical classification

In [123]:
x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = tf.keras.layers.Dense(units=2, activation="softmax")(x)

model = tf.keras.Model(inputs=base_model.input, outputs=outputs)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

+ create `ImageDataGenerator()` instance separately for training a validation set
+ for training dataset, we can set several data augmentation parameters such as rotation, horizontal flip etc. -> it is not generally advised to do the same also for validation dataset, therefore, for validation only neccessary rescaling is set

In [124]:
train_datagen = ImageDataGenerator(
    rotation_range=15,
    rescale=1./255
)

validation_datagen = ImageDataGenerator(
    rescale=1./255
)

+ create iterator through training and validation dataset with `.flow_from_directory()`

In [125]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle = True)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical', 
    shuffle=True)


Found 435 images belonging to 2 classes.
Found 110 images belonging to 2 classes.


In [126]:
history = model.fit(train_generator,
                    epochs=EPOCHS,
                    validation_data=validation_generator)

Epoch 1/5
 8/14 [================>.............] - ETA: 6s - loss: 0.9307 - accuracy: 0.5273

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


14/14 [==============================] - 21s 2s/step - loss: 0.8196 - accuracy: 0.5885 - val_loss: 0.5301 - val_accuracy: 0.7091
Epoch 2/5
14/14 [==============================] - 18s 1s/step - loss: 0.4677 - accuracy: 0.7885 - val_loss: 0.4222 - val_accuracy: 0.7818
Epoch 3/5
14/14 [==============================] - 18s 1s/step - loss: 0.3561 - accuracy: 0.8391 - val_loss: 0.3563 - val_accuracy: 0.8545
Epoch 4/5
14/14 [==============================] - 18s 1s/step - loss: 0.2753 - accuracy: 0.8736 - val_loss: 0.3375 - val_accuracy: 0.8727
Epoch 5/5
14/14 [==============================] - 19s 1s/step - loss: 0.2486 - accuracy: 0.8897 - val_loss: 0.3279 - val_accuracy: 0.8727


## Export to TSjs

In [65]:
!pip install tensorflowjs

     |████████████████████████████████| 71kB 5.5MB/s 
     |████████████████████████████████| 112kB 16.6MB/s 
  Found existing installation: tensorflow-hub 0.10.0
    Uninstalling tensorflow-hub-0.10.0:
      Successfully uninstalled tensorflow-hub-0.10.0


In [127]:
import tensorflowjs as tfjs
tfjs.converters.save_keras_model(model, "/content/drive/My Drive/assignment05_data/export_model")

/usr/local/lib/python3.6/dist-packages/tensorflowjs/converters/keras_h5_conversion.py:123: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  return h5py.File(h5file)


In [113]:
import shutil
shutil.make_archive("export_model", 'zip', "/content/drive/My Drive/assignment05_data/export_model/", )

from google.colab import files
files.download('export_model.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>